## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pyspark.sql.functions import * 
from pyspark.sql.window import Window

# File location and type
file_location = "/FileStore/tables/online_retail_II.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df.take(10))

Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01T07:45:00.000+0000,6.95,13085.0,United Kingdom
489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01T07:45:00.000+0000,6.75,13085.0,United Kingdom
489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01T07:45:00.000+0000,6.75,13085.0,United Kingdom
489434,22041,"""RECORD FRAME 7"""" SINGLE SIZE """,48,2009-12-01T07:45:00.000+0000,2.1,13085.0,United Kingdom
489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01T07:45:00.000+0000,1.25,13085.0,United Kingdom
489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01T07:45:00.000+0000,1.65,13085.0,United Kingdom
489434,21871,SAVE THE PLANET MUG,24,2009-12-01T07:45:00.000+0000,1.25,13085.0,United Kingdom
489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01T07:45:00.000+0000,5.95,13085.0,United Kingdom
489435,22350,CAT BOWL,12,2009-12-01T07:46:00.000+0000,2.55,13085.0,United Kingdom
489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01T07:46:00.000+0000,3.75,13085.0,United Kingdom


In [0]:
retail_df = df.cache()
display(retail_df.take(5))

Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01T07:45:00.000+0000,6.95,13085.0,United Kingdom
489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01T07:45:00.000+0000,6.75,13085.0,United Kingdom
489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01T07:45:00.000+0000,6.75,13085.0,United Kingdom
489434,22041,"""RECORD FRAME 7"""" SINGLE SIZE """,48,2009-12-01T07:45:00.000+0000,2.1,13085.0,United Kingdom
489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01T07:45:00.000+0000,1.25,13085.0,United Kingdom


### Total Invoice Distribution

In [0]:
# function to calculate mean, minimum, maximum, median, mode
def show_distribution(df, col):
  df.select(min(df[col])).show()
  df.select(max(df[col])).show()
  df.select(mean(df[col])).show()
  print(df.approxQuantile(col,[0.5],0))
  print(df.groupby(col).count().orderBy(desc('count')).first())
  
# creating a Dataframe containing invoice amounts, using the original df and multiplying the according columns
# negative results are not of interest for this (they represent cancellations) so they are filtered out

invoice_amount_df = retail_df.withColumn('Amount', retail_df['Quantity'] * retail_df['Price']).groupBy('Invoice').agg(sum('Amount').alias('Amount')).filter("Amount > 0.0")
show_distribution(invoice_amount_df, 'Amount')

+-----------+
min(Amount)|
+-----------+
 0.19|
+-----------+

+-----------+
max(Amount)|
+-----------+
 168469.6|
+-----------+

+-----------------+
 avg(Amount)|
+-----------------+
523.3037611158244|
+-----------------+

[304.3100000000002]
Row(Amount=15.0, count=118)

In [0]:
# show distribution of first 85 quantiles, to remove outliers
quantiles = invoice_amount_df.approxQuantile('Amount', [0.0, 0.85], 0)
filtered_amount_df = invoice_amount_df.filter(col("Amount") > quantiles[0]).filter(col("Amount") < quantiles[1])
show_distribution(filtered_amount_df, 'Amount')

+-----------+
min(Amount)|
+-----------+
 0.38|
+-----------+

+-----------------+
 max(Amount)|
+-----------------+
724.2499999999999|
+-----------------+

+-----------------+
 avg(Amount)|
+-----------------+
271.6918611771613|
+-----------------+

[256.24999999999994]
Row(Amount=15.0, count=118)

### Monthly Placed and canceled Orders

In [0]:
# new colum for "yyyymm"
orders_df = retail_df
yyyymm_df = orders_df.select('Invoice', 'InvoiceDate', year("InvoiceDate").alias("Year"), month("InvoiceDate").alias("Month"))
yyyymm_df = yyyymm_df.withColumn('yyyymm', yyyymm_df['Year']*100+yyyymm_df['Month'])
yyyymm_df = orders_df.join(yyyymm_df,['Invoice','InvoiceDate'], how ='right_outer')
yyyymm_df = yyyymm_df.orderBy('yyyymm')
orders_df = yyyymm_df.groupBy('yyyymm').agg(countDistinct('Invoice').alias('Orders'))

#get canceled order for each month
canceled_df = yyyymm_df.filter(col('Invoice').startswith('C')).groupBy('yyyymm').agg(countDistinct('Invoice').alias('Canceled'))

#calculated placed orders
orders_df = orders_df.join(canceled_df, 'yyyymm', how = 'inner')
orders_df = orders_df.withColumn('placed', orders_df['Orders']-2*orders_df['Canceled'])

display(orders_df.orderBy('yyyymm'))

yyyymm,Orders,Canceled,placed
200912,2330,401,1528
201001,1633,300,1033
201002,1969,240,1489
201003,2367,407,1553
201004,1892,304,1284
201005,2418,407,1604
201006,2216,357,1502
201007,2017,344,1329
201008,1877,273,1331
201009,2375,371,1633


###Monthly Sales

In [0]:
monthly_sales_df = yyyymm_df.withColumn('Amount', yyyymm_df['Quantity'] * yyyymm_df['Price']).groupBy("yyyymm").agg(sum("Amount").alias('Amount'))
monthly_sales_df = monthly_sales_df.withColumn('Amount',round(monthly_sales_df['Amount'], 2))
display(monthly_sales_df)

yyyymm,Amount
200912,7.74710379E7
201001,4.335743827E7
201002,2.219918002E7
201003,3.867888423E7
201004,2.654554476E7
201005,2.689071672E7
201006,3.344102679E7
201007,2.296480636E7
201008,2.772266082E7
201009,3.279894642E7


###Monthly Sales Growth

In [0]:
monthly_growth_window = Window.orderBy('yyyymm')
sale_growth_df = monthly_sales_df.withColumn('previous', lag(monthly_sales_df['Amount']).over(monthly_growth_window))
sale_growth_df = sale_growth_df.withColumn('growth',when(isnull((sale_growth_df['Amount']-sale_growth_df['previous'])/sale_growth_df['previous']),0.0).otherwise((sale_growth_df['Amount']-sale_growth_df['previous'])/sale_growth_df['previous']))

sale_growth_df = sale_growth_df.withColumn('previous',round(sale_growth_df['previous'],2))
sale_growth_df = sale_growth_df.withColumn('growth',round(sale_growth_df['growth'],2))
display(sale_growth_df)

yyyymm,Amount,previous,growth
200912,7.74710379E7,null,0.0
201001,4.335743827E7,7.74710379E7,-0.44
201002,2.219918002E7,4.335743827E7,-0.49
201003,3.867888423E7,2.219918002E7,0.74
201004,2.654554476E7,3.867888423E7,-0.31
201005,2.689071672E7,2.654554476E7,0.01
201006,3.344102679E7,2.689071672E7,0.24
201007,2.296480636E7,3.344102679E7,-0.31
201008,2.772266082E7,2.296480636E7,0.21
201009,3.279894642E7,2.772266082E7,0.18


### Monthly Active Users

In [0]:
#get monthly active users
active_users_df = yyyymm_df.groupBy('yyyymm').agg(countDistinct('Customer ID').alias('Active_user'))
display(active_users_df)

yyyymm,Active_user
200912,1045
201001,786
201002,807
201003,1111
201004,998
201005,1062
201006,1095
201007,988
201008,964
201009,1202


### New and Existing Users

In [0]:
#get the new user
new_user_df = yyyymm_df.groupBy('Customer ID').agg(first('yyyymm').alias('yyyymm')).groupby('yyyymm').agg(countDistinct('Customer ID').alias('New_user'))
# get the total user
total_user_df = yyyymm_df.groupBy('yyyymm').agg(countDistinct('Customer ID').alias('Total_user'))
total_user_df=total_user_df.join(new_user_df,'yyyymm',how ='left_outer')
total_user_df = total_user_df.withColumn('Return_user', total_user_df['Total_user']-total_user_df['New_user'])

display(total_user_df)

yyyymm,Total_user,New_user,Return_user
201108,980,106,874
201011,1683,322,1361
201101,783,71,712
201004,998,291,707
201003,1111,436,675
201103,1020,178,842
201112,686,28,658
201012,948,77,871
201001,786,394,392
201005,1062,254,808


### Finding RFM

In [0]:
# find most recent invoice date for each customer, then find days since the invoice was processed
Today = pd.to_datetime('today')
retail_df = retail_df.orderBy('InvoiceDate')
recency_df = retail_df.groupBy('Customer Id').agg(datediff(lit(Today), max('InvoiceDate')).alias('Recency'))

# count invoices associated with each customer
frequency_df = retail_df.groupBy('Customer Id').agg(countDistinct('Invoice').alias('Frequency'))

# calculate how much each user spent
monetary_df = retail_df.withColumn('Amount', retail_df['Quantity'] * retail_df['Price']).groupBy('Customer Id').agg(sum('Amount').alias('Amount'))
monetary_df = monetary_df.withColumn('Amount',round(monetary_df['Amount'],2))

# join results
rfm_df = recency_df.join(frequency_df, 'Customer Id', 'left_outer')
rfm_df = rfm_df.join(monetary_df, 'Customer Id',how = 'left_outer')
display(rfm_df.orderBy('Customer Id').take(20))

Customer Id,Recency,Frequency,Amount
null,3325,null,null
12346.0,3650,17,-64.68
12347.0,3327,8,5633.32
12348.0,3400,5,2019.4
12349.0,3343,5,4404.54
12350.0,3635,1,334.4
12351.0,3700,1,300.93
12352.0,3361,13,1889.21
12353.0,3529,2,406.76
12354.0,3557,1,1079.4


### RFM Segmentation